In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.expanduser('~/clinical_transformer/'))
sys.path.append(os.path.expanduser('~/samecode/'))
sys.path.append(os.path.expanduser('~/python/usr/local/lib/python3.6/dist-packages/'))
sys.path.append(os.path.expanduser('~/pbmf/'))

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.auto import tqdm
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
%config InlineBackend.figure_format = 'png'
from collections import Counter

import matplotlib 
matplotlib.rcParams['figure.dpi'] = 300

import matplotlib.pyplot as plt
from collections import Counter


In [4]:
from samecode.plot.pyplot import subplots
from samecode.survival.plot import KMPlot
from samecode.random import set_seed

# Retrieve data

In [5]:
datasets = {}

for split in ['train', 'test']:    
    # 1 OAK
    train = pd.read_csv('../../OAK_POPLAR/data/data_{}_benchmark.csv'.format(split))
    train['ARM'] = train.Treatment.replace(['SOC', 'IO'], ['Docetaxel', 'Atezolizumab'])
    train['pruning_risk'] = train['pruning_consensus_risk']
    train['time'] = train['OS']
    train['event'] = train['OS_EVENT']
    datasets[('oak', split)] = train
    
    #2 JAVELIN
    train = pd.read_csv('../../JAVELIN_renal_101_trial/data/data_{}.csv'.format(split))
    train['pruning_risk'] = train['pruning_consensus_risk']
    train['time'] = train['PFS_P']
    train['event'] = train['PFS_EVENT']
    datasets[('javelin', split)] = train
    
    #3 CHECKMATE
    train = pd.read_csv('../../Braun2020_Checkmate/data/data_{}-benchmark.csv'.format(split))
    train['ARM'] = train.Arm.replace(['EVEROLIMUS', 'NIVOLUMAB'], ['Everolimus', 'Nivolumab'])
    train['pruning_risk'] = train['pruning_consensus_risk']
    train['time'] = train['OS']
    train['event'] = train['OS_CNSR']
    datasets[('checkmate', split)] = train
        
    #5 IMVIGOR211 210-211
    train = pd.read_csv('../../IMVIGOR211/data/data_{}-ph2-ph3.csv'.format(split))
    train['OS_TIME'] = train['OS_TIME'] / 30.44
    train['pruning_risk'] = train['pruning_consensus_risk_min']
    train['time'] = train['OS_TIME']
    train['event'] = train['OS_EVENT']
    datasets[('imvigor', split)] = train
        
    #6 IMMotion 150
    train = pd.read_csv('../../IMMotion150/data/data_{}.csv'.format(split))
    train['OS_TIME'] = train['OS_TIME'] / 30.44
    train['pruning_risk'] = train['pruning_consensus_risk_min']
    train['time'] = train['OS_TIME']
    train['event'] = train['OS_EVENT']
    datasets[('immotion', split)] = train
    

    #8 BRCA
    # prunning did not improve model performance in training set.
    train = pd.read_csv('../../BRCA/data/data_{}_benchmark.csv'.format(split))
    train['ARM'] = train.Treatment.replace(['CT', 'HT'], ['Chemo', 'Hormone'])
    train['pruning_bin_risk'] = train['bin_risk_selected_0.95']
    train['pruning_risk'] = train['consensus_risk_selected_0.95']
    train['time'] = train['time']
    train['event'] = train['death']
    datasets[('brca', split)] = train
    
    
    #9 RETINOPATHY
    train = pd.read_csv('../../Retinopathy/data/retinopathy_bin_{}_results.csv'.format(split))
    train['pruning_risk'] = train['pruning_consensus_risk']
    train['time'] = train['time']
    train['event'] = train['event']
    datasets[('retinopathy', split)] = train
    

In [6]:
import pickle
pickle.dump(datasets, open('../data/dataset.pk', 'wb'))